In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all' # to print multiple outputs from the same cell
import math
import utils
import numpy as np
import pandas as pd
import scipy.stats as stats
import matplotlib.pyplot as plt
import seaborn as sns
from operator import index
from collections import defaultdict
from scipy.stats import pearsonr
from datetime import datetime

In [2]:
users_df = pd.read_csv("dataset/users.csv")
#tweets_df = pd.read_csv("dataset/tweets.csv", usecols=["id", "user_id"]) # read only this two colums (saving space)
tweets_df = pd.read_csv("dataset/tweets.csv")

# Data Understanding and Preparation

## Users Data

In users.csv there are the following variables:
1. User Id: a unique identifier of the user
2. Statues Count: According to the teacher, this is the count of the tweets made by the user at the moment of data
crawling. According to [Twitter API docs](https://developer.twitter.com/en/docs/twitter-api/v1/data-dictionary/object-model/user), this is the number of Tweets (including retweets) issued by the user, but not replies (according to Francesca Naretto); since tweets.csv inclues also users' replies note that **there is no link between the number of tweets for each user in tweets.csv and statuses_count**.
3. Lang: the user’s language selected
4. Created at: the timestamp in which the profile was created
5. Label: a binary variable that indicates if a user is a bot or a genuine user

In [ ]:
users_df.info(verbose=True, show_counts=True, memory_usage= "deep")

### Attribute type and quality

In the **user** dataset there are 6 columns:

1. The id **column** seems to be ok, all values are integer and there are not null values, we have to check possible duplicates
 
2. We have 1 null value in the **name** column, we also assume that the name could be a string, a number or a special character, the names are not necessarily unique, but maybe it's intresting to check the frequency distribution.

3. In the **lang** column we don't have null values, but we have to check whether there are problems in the pattern used to express the language, we expect a categorical attribute 

4. The **bot** column is numerical as expected (binary), we have to check whether all the numbers are 0 or 1

5. The attribute **created_at** has no null values, but we have to check the correctness of the date, both sintactic and semantic (not too far in the past or in the future)

6. The **status_count** column has 399 of null values, in the non-null values there would semm to be unexpected float

Let's verify the uniqueness of ids: all the ids are unique!

In [6]:
print("Number of total IDs:", len(users_df["id"]))
print("Number of unique IDs:", len(pd.unique(users_df["id"])))

Number of total IDs: 11508
Number of unique IDs: 11508


As said before one name is null. There are also duplicate names, but this isn't a surprising behaviour and by plotting the names' frequencies we can see that there aren't strange phenomena.

In [ ]:
print("Number of total names:", len(users_df["name"]))
print("Number of unique names:", len(pd.unique(users_df["name"])))

freq = {}
for n in users_df['name']:
    if n in freq:
        freq[n] += 1
    else:
        freq[n] = 1

number_of_total_names = len(users_df["name"])
not_empty_or_missing_names = []
empty_or_missing_names = []
names_with_only_spaces = []

# iterate over all names looking for errors
for value in users_df["name"]:
    if pd.isna(value) or value == "": # name is nan or is_empty string
        empty_or_missing_names.append(value)
    if str(value).strip() == "":
            names_with_only_spaces.append(value)
    elif not(pd.isna(value) or value == ""):
        not_empty_or_missing_names.append(value)
        
print(f"Number of total names = {number_of_total_names} vs total name values that are not NA or empty = {len(not_empty_or_missing_names)}")
print(f"Number of total names = {number_of_total_names} vs total name values that are NA or empty = {len(empty_or_missing_names)}")

pd.DataFrame({"frequencies": [_ for _ in freq.values()]}).hist(
    column=["frequencies"], 
    log=True, 
    bins=utils.get_sturges_bins(len(freq.values()))
)

Now let's check the lang column

In [ ]:
pd.unique(users_df["lang"])  

The "lang" field is composed of [IETF language codes](https://en.wikipedia.org/wiki/IETF_language_tag). By selecting only the unique values it's possible to see that there are some erroneous values:
* "Select Language..." and "xx-lc" seems to be **default values**
* other values are not properly correct (e.g. "zh-cn" instead of "zh-CN")
We propose to check the most common language used by these 'erroneous values' users and provide them with a more fitting language attribute.

In [ ]:
utils.repair_lang_attribute(users_df)
pd.unique(users_df["lang"])

Since wrong values are just the 0.02% of the number of rows they are just dropped, while the other values are mapped to the correct ones.

The bot attribute is perfectly as expected, all non-null binary values

In [ ]:
pd.unique(users_df["bot"])

We observe that the created_at coloumn is recognized by pandas as an object, and not as a datetime as we would expect from this attribute. Clean created_at field, by converting string to datetime

In [ ]:
# parsing string to datetime obj
users_df["created_at"] = pd.to_datetime(users_df["created_at"])

We expect the statuses count to be an integer, but pandas has interpreted it as a float. This is probably due to the presence of NaN values

In [7]:
users_df.statuses_count = users_df.statuses_count.apply(pd.to_numeric, errors="coerce").astype({"statuses_count": "Int64"})

In [8]:
users_df.info(verbose=True, show_counts=True, memory_usage= "deep")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11508 entries, 0 to 11507
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   id              11508 non-null  int64 
 1   name            11507 non-null  object
 2   lang            11508 non-null  object
 3   bot             11508 non-null  int64 
 4   created_at      11508 non-null  object
 5   statuses_count  11109 non-null  Int64 
dtypes: Int64(1), int64(2), object(3)
memory usage: 2.5 MB


### Distribution of variables and statistics
Let's study them!

In [ ]:
users_df.hist(
    column=["statuses_count"], 
    log=True, 
    bins=utils.get_sturges_bins(len(users_df["statuses_count"]))
)

users_df.hist(
    column=["statuses_count"], 
    by="bot", 
    log=True,
    bins=utils.get_sturges_bins(len(users_df["statuses_count"])) #FIX THIS: USES ALL THE SAMPLES, NOT JUST THE BOTS AND THE USERS
)

In [ ]:
langs = pd.unique(users_df["lang"]) 
bot_freqs = []
user_freqs = []
for lang in langs:
    user_freqs.append(len(users_df.query(f"lang == '{lang}' & bot == 0")))
    bot_freqs.append(len(users_df.query(f"lang == '{lang}' & bot == 1")))
langs_df = pd.DataFrame({"lang": langs, "bot_freqs": bot_freqs, "user_freqs": user_freqs})
langs_df.plot.bar(x="lang", logy=True)

## Tweets data

In tweets.csv each row contains information about a single tweet. In this case the variables
are:
1. ID: a unique identifier for the tweet
2. User Id: a unique identifier for the user who wrote the tweet
3. Retweet count: number of retweets for the tweet in analysis
4. Reply count: number of reply for the tweet in analysis
5. Favorite count: number of favorites (likes) received by the tweet
6. Num hashtags: number of hashtags used in the tweet
7. Num urls: number of urls in the tweet
8. Num mentions: number of mentions in the tweet
9. Created at: when the tweet was created
10. Text: the text of the tweet

Regarding the num * fields, we don't have to check the validity of the values and can assume they are correct, except for the null and clear incorrect values. In order to substitute the null values of these fields we can exploit the information of the text (however for the mentions and urls it is impossible to check their validity because we may have a mention to a user that does not exist and we cannot know it)

### Assessing data quality

In [3]:
tweets_df.info(verbose=True, show_counts=True, memory_usage= "deep")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13664696 entries, 0 to 13664695
Data columns (total 10 columns):
 #   Column          Non-Null Count     Dtype 
---  ------          --------------     ----- 
 0   id              13664694 non-null  object
 1   user_id         13447413 non-null  object
 2   retweet_count   13227562 non-null  object
 3   reply_count     13016818 non-null  object
 4   favorite_count  13017154 non-null  object
 5   num_hashtags    12607172 non-null  object
 6   num_urls        13016073 non-null  object
 7   num_mentions    12810531 non-null  object
 8   created_at      13664696 non-null  object
 9   text            13126975 non-null  object
dtypes: object(10)
memory usage: 8.9 GB


keeping only the tweets with user_id in user dataset

In [14]:
def get_tweets_id_with_invalid_user(tweets_df):
    tmp = pd.to_numeric(tweets_df["user_id"], errors="coerce")
    ids_are_not_in_users_df = tmp[tmp.isin(users_df["id"]) == False]
    return ids_are_not_in_users_df

invalid_users = get_tweets_id_with_invalid_user(tweets_df)

tweets_df.drop(invalid_users.index, inplace=True)
tweets_df.info(verbose=True)

del invalid_users

#pd.to_numeric(tweets_df["user_id"], errors="coerce").info(verbose=True, show_counts=True)

""

TypeError: Series.dropna() got an unexpected keyword argument 'subset'

clean id field by first removing nan values (just 2), then tring to cast to int and removing duplicates

In [ ]:
#tweets_df.dropna(subset=["id"], inplace=True)

# removing not numeric strings
#pd.to_numeric(tweets_df["id"])

#tweets_df["id"].isin(users)

From the method above we observe that all our atributes except for "created_at" have one or more elements with null values.

In [ ]:
"""
wrong_ids = []
for (i,k) in enumerate(df["id"]):
    if not isinstance(k, str) or not k.isnumeric():
        wrong_ids.append(i)
print(len(wrong_ids)/len(df["id"]))
""" 

Checks if all the tweets were created after the first tweet published on twitter (so we don't have something strange like a tweet created in 01-01-1990)

In [ ]:
# checks if all the users were created after the first tweet published on twitter (so we don't have something strange like a user created in 01-01-1990)
twitter_first_tweet_datetime = datetime(2006,3,21,12,50,0)
#string_to_datetime = lambda string: datetime.strptime(string, expected_format)
published_after_twitter_first_tweet = lambda x: x > twitter_first_tweet_datetime
all(map(published_after_twitter_first_tweet, users_df["created_at"]))


### Distribution of variables and statistics
Let's study them!

In [ ]:
# should we use sturgen rule for number of bins?

# give error: ValueError: hist method requires numerical or datetime columns, nothing to plot.
#tweets_df.hist(column=["reply_count","retweet_count", "favorite_count", "num_hashtags", "num_urls", "num_mentions"])

In [ ]:
#tweets_df.describe()

### Variables trasformations (?)

### Pairwise correlations